missing deps
- deepspeed
- plotly
- nbformat

In [ ]:
import re
import tqdm
import numpy as np
import pandas as pd
import plotly.express as px
from functools import reduce
from openminers.text_to_text.dummy.test_template import TestMiner

In [2]:
miner = TestMiner()
for i in range(3):
    print(f'{"---"*30}\n{i}:\n{miner.model("")}')

------------------------------------------------------------------------------------------ 
 Interesting question.
In order to address that context it's probably useful to tentatively medidate on the contextual viewpoints of the context.
------------------------------------------------------------------------------------------ 
 I see your point
In order to appropriately answer that context it's probably intelligent to simply medidate on the subjective value of the context.
------------------------------------------------------------------------------------------ 
 Well now then..
In order to appropriately answer that question it's terribly valuable to first examine the subjective utility of the question.
------------------------------------------------------------------------------------------ 
 I've been waiting for this moment.
In order to nail that query it's positively important to simply ruminate over the spiritual essence of the query.
-------------------------------------------

In [10]:
nunique = reduce(lambda x,y: x*y, [len(words) for words in miner.choices.values()],1)
print(f'Number of unique completions is {nunique:,}')

Number of unique completions is 350,269,920


In [18]:
def substr_freq_reward(completion, sub='a'):
    # this reward model loves the letter a
    return completion.count(sub)/len(completion)*10

def word_length_reward(completion):
    return np.mean([len(word) for word in completion.split()])*0.5


miner = TestMiner()
miner.learning_rate=0.1

messages = [
    {'role':'system', 'content':'You are a good boy'},
    {'role':'user', 'content':'What is the capital of Texas?'}
]
results = []
scores = []
responses = []
max_no_change = 10
pbar = tqdm.tqdm(range(10000))
for i in pbar:
    # print('--'*40)
    response = miner.forward(messages=messages)
    responses.append(response)
    
    if i>max_no_change and len(set(responses[-max_no_change:]))==1:
        print(f'Response has not changed for {max_no_change} steps... quitting training')
        break
    
    rewards = substr_freq_reward(response, sub='you')    
    # rewards = word_length_reward(response)
    # print(f'response = {response!r}')
    # print(f'rewards = {rewards:.3f}')
    miner.backward(messages=messages, response=response, rewards=rewards)
    scores.append(rewards)
    if i%10 == 0:
        pbar.set_description(f'Rewards: {rewards:.3f}')
    
    for key, words in miner.choices.items():
        results.append(pd.DataFrame({'step':i, 'response':response, 'rewards':rewards, 'key':key, 'words':words,'weights':miner.weights[key]}))


Rewards: 0.078: 100%|██████████| 10000/10000 [00:48<00:00, 207.58it/s]


In [19]:

px.line(pd.Series(scores, name='Reward'),y='Reward').show()


In [20]:
df = pd.concat(results)
df

In [ ]:
# plot weights during training
for key in miner.choices:
    df_sub = df.loc[df['key']==key].copy()
    df_sub['weights_moving_avg'] = df_sub.weights.rolling(10).mean()
    px.line(df_sub,x='step',y='weights_moving_avg',color='words',
            title=key, template='plotly_white',
            ).update_traces(opacity=0.4).show()
